### Testing for Interactive use case

In [2]:
# pip install --upgrade ray-on-aml

In [2]:
import mlflow
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from IPython.display import clear_output
import time
import platform
import sys
# sys.path.append("../") # go to parent dir
import importlib


In [3]:
#You can pre-provision "worker-cpu-v3" in the same vnet with your compute instance
from ray_on_aml.core import Ray_On_AML
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="worker-cpu-v3", additional_pip_packages=['torch==1.10.0'])
_, ray = ray_on_aml.getRay()


rank returned is  None
rank returned is  None
azureml_py38


2021-12-14 23:51:22,372	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265


Found existing cluster, use it.


2021-12-14 23:51:38,353	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 10.0.0.13:6379
2021-12-14 23:51:38,486	WARNING worker.py:1219 -- The autoscaler failed with the following error:
Terminated with signal 15
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 543, in <module>
    monitor.run()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 439, in run
    self._run()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/autoscaler/_private/monitor.py", line 346, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



Waiting: Cluster status is in  Queued
Waiting: Cluster status is in  Queued
Waiting: Cluster status is in  Queued
Waiting: Cluster status is in  Queued


In [5]:
ray.cluster_resources()

{'CPU': 26.0,
 'node:10.0.0.21': 1.0,
 'object_store_memory': 37732648548.0,
 'memory': 82534642896.0,
 'node:10.0.0.18': 1.0,
 'node:10.0.0.20': 1.0,
 'node:10.0.0.17': 1.0,
 'node:10.0.0.22': 1.0,
 'GPU': 1.0,
 'accelerator_type:K80': 1.0,
 'node:10.0.0.13': 1.0}

### Dask on Ray

In [39]:
import ray
ray.init()
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
dask.config.set(scheduler=ray_dask_get)
d_arr = da.from_array(np.random.randint(0, 1000, size=(256, 256)))

# The Dask scheduler submits the underlying task graph to Ray.
d_arr.mean().compute(scheduler=ray_dask_get)

# Set the scheduler to ray_dask_get in your config so you don't have to
# specify it on each compute call.

df = dd.from_pandas(
    pd.DataFrame(
        np.random.randint(0, 10000, size=(1024, 2)), columns=["age", "grade"]),
    npartitions=2)
df.groupby(["age"]).mean().compute()

# ray.shutdown()

2021-12-13 05:32:12,050	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8266
2021-12-13 05:32:12,054	WARNING services.py:1816 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 5310255104 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


,grade
age,
6,4318.0
45,885.0
46,6405.0
77,6895.0
81,8404.0
...,...
9895,8000.0
9915,7781.0
9951,7748.0


In [34]:
#dask

# import ray
from ray.util.dask import ray_dask_get
import dask
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime

from azureml.core import Workspace, Dataset, Model
from adlfs import AzureBlobFileSystem
# account_key = ws.get_default_keyvault().get_secret("adls7-account-key")
# account_name="adlsgen7"
# abfs = AzureBlobFileSystem(account_name="adlsgen7",account_key=account_key,  container_name="mltraining")
abfs2 = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")


storage_options={'account_name': account_name, 'account_key': account_key}

# ddf = dd.read_parquet('az://mltraining/ISDWeatherDelta/year2008', storage_options=storage_options)

data = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2012/"], filesystem=abfs2)
data1 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2015/"], filesystem=abfs2)
data2 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2010/"], filesystem=abfs2)
data3 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2009/"], filesystem=abfs2)
data4 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2011/"], filesystem=abfs2)
data5 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2013/"], filesystem=abfs2)
data6 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2014/"], filesystem=abfs2)



Metadata Fetch Progress: 100%|██████████| 16/16 [00:04<00:00,  3.42it/s]


In [35]:
all_data =data.union(data1).union(data2).union(data3).union(data4).union(data5).union(data6)
all_data.count()

690211698

In [36]:
start = time.time()
#convert Ray dataset to Dask dataframe 
all_data_dask = all_data.to_dask().describe().compute()
print(all_data_dask)
stop = time.time()
print("duration ", (stop-start))
#717s for single machine nc6
#duration  1854.3798034191132 cluster 1st time


(dask:('quantiles-1-d7cd6e40aa11b038fe5ca5a773d25ac8', 298) pid=906, ip=10.0.0.22) 


(dask:('from-delayed-37bcded3107f2c11146104d14a59d0ad', 368) pid=80603) 


(dask:('quantiles-1-6e697189017d81f71030f5e8bf0ee8a2', 189) pid=263, ip=10.0.0.22) 
(dask:('quantiles-1-040e133c45167fb786e4c5b3a28faf1c', 66) pid=905, ip=10.0.0.22) 


(dask:block_to_df-c2e390a0-5836-4fea-82e3-19ef23ae7d25 pid=198, ip=10.0.0.22) 


(dask:('dropna-ba9cf0277682a5953a58639127dd84bd', 192) pid=2029, ip=10.0.0.20) 


...

(dask:('quantiles-1-046f250e7af81064a522328b34088f5b', 410) pid=29527, ip=10.0.0.20) 
(dask:('dropna-d24804d88d961fea6af2f5601cea6261', 493) pid=2574, ip=10.0.0.17) 
(dask:('dropna-8598322c3d00cd5ed4009f78d212d981', 303) pid=2574, ip=10.0.0.17) 


(dask:('from-delayed-37bcded3107f2c11146104d14a59d0ad', 429) pid=374, ip=10.0.0.17) 


(dask:('quantiles-1-24c4de00c62d75056557559435285835', 561) pid=2574, ip=10.0.0.17) 


(dask:('dropna-d4e810db3c5e478a2250add67fbce6c8', 506) pid=200, ip=10.0.0.17) 


           latitude     longitude     elevation     windAngle     windSpeed  \
count  6.902117e+08  6.902117e+08  6.902117e+08  5.414428e+08  5.624939e+08   
mean   3.796754e+01 -4.137369e+01  4.173534e+02  1.642972e+02  3.452701e+00   
std    2.068093e+01  7.837449e+01  6.445339e+02  1.146524e+02  2.888769e+00   
min   -9.000000e+01 -1.799830e+02 -3.500000e+02  0.000000e+00  0.000000e+00   
25%    3.395600e+01 -9.571000e+01  5.000000e+01  8.000000e+01  2.000000e+00   
50%    4.155400e+01 -7.619200e+01  2.100000e+02  1.900000e+02  3.100000e+00   
75%    4.918300e+01  1.668300e+01  5.500000e+02  2.800000e+02  5.700000e+00   
max    8.733300e+01  9.999990e+02  9.999000e+03  3.600000e+02  9.000000e+01   

        temperature  seaLvlPressure  presentWeatherIndicator  \
count  6.779744e+08    2.284912e+08             6.001119e+07   
mean   1.185499e+01    1.014672e+03             2.821610e+01   
std    1.247398e+01    9.456261e+00             2.883805e+01   
min   -8.860000e+01    8.727000e

### Ray Tune for distributed ML tunning

In [22]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
# import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total
def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        tune.report(mean_accuracy=acc)

        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")
search_space = {
    "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
    "momentum": tune.uniform(0.01, 0.09)
}

# Uncomment this to enable distributed execution
# ray.shutdown()
# ray.init(address="auto",ignore_reinit_error=True)
# ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True,ignore_reinit_error=True )
# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

analysis = tune.run(train_mnist, config=search_space)


In [3]:
 import sklearn.datasets
 import sklearn.metrics
 from sklearn.model_selection import train_test_split
 import xgboost as xgb

 from ray import tune


 def train_breast_cancer(config):
     # Load dataset
     data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
     # Split into train and test set
     train_x, test_x, train_y, test_y = train_test_split(
         data, labels, test_size=0.25)
     # Build input matrices for XGBoost
     train_set = xgb.DMatrix(train_x, label=train_y)
     test_set = xgb.DMatrix(test_x, label=test_y)
     # Train the classifier
     results = {}
     xgb.train(
         config,
         train_set,
         evals=[(test_set, "eval")],
         evals_result=results,
         verbose_eval=False)
     # Return prediction accuracy
     accuracy = 1. - results["eval"]["error"][-1]
     tune.report(mean_accuracy=accuracy, done=True)


 config = {
     "objective": "binary:logistic",
     "eval_metric": ["logloss", "error"],
     "max_depth": tune.randint(1, 9),
     "min_child_weight": tune.choice([1, 2, 3]),
     "subsample": tune.uniform(0.5, 1.0),
     "eta": tune.loguniform(1e-4, 1e-1)
 }
 analysis = tune.run(
     train_breast_cancer,
     resources_per_trial={"cpu": 1},
     config=config,
     num_samples=10)


### Distributed XGBoost https://docs.ray.io/en/latest/xgboost-ray.html

In [18]:
# pip install xgboost_ray 

In [14]:
from xgboost_ray import RayXGBClassifier, RayParams
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

seed = 42

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.25, random_state=42
)

clf = RayXGBClassifier(
    n_jobs=10,  # In XGBoost-Ray, n_jobs sets the number of actors
    random_state=seed
)

# scikit-learn API will automatically conver the data
# to RayDMatrix format as needed.
# You can also pass X as a RayDMatrix, in which case
# y will be ignored.

clf.fit(X_train, y_train)

pred_ray = clf.predict(X_test)
print(pred_ray.shape)

pred_proba_ray = clf.predict_proba(X_test)
print(pred_proba_ray.shape)

# It is also possible to pass a RayParams object
# to fit/predict/predict_proba methods - will override
# n_jobs set during initialization

clf.fit(X_train, y_train, ray_params=RayParams(num_actors=10))

pred_ray = clf.predict(X_test, ray_params=RayParams(num_actors=10))
print(pred_ray.shape)


2021-12-15 00:02:37,433	INFO main.py:971 -- [RayXGBoost] Created 10 new actors (10 total actors). Waiting until actors are ready for training.
2021-12-15 00:02:39,774	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=19133) [00:02:39] task [xgboost.ray]:140672750021792 got new rank 0
(_RemoteRayXGBoostActor pid=19223) [00:02:39] task [xgboost.ray]:140510980246064 got new rank 1
(_RemoteRayXGBoostActor pid=582, ip=10.0.0.17) [00:02:39] task [xgboost.ray]:139840916503568 got new rank 2
(_RemoteRayXGBoostActor pid=736, ip=10.0.0.18) [00:02:40] task [xgboost.ray]:140534154546144 got new rank 4
(_RemoteRayXGBoostActor pid=737, ip=10.0.0.18) [00:02:39] task [xgboost.ray]:140264658713568 got new rank 3
(_RemoteRayXGBoostActor pid=643, ip=10.0.0.20) [00:02:40] task [xgboost.ray]:140177723526160 got new rank 5
(_RemoteRayXGBoostActor pid=642, ip=10.0.0.20) [00:02:40] task [xgboost.ray]:140138270759952 got new rank 6
(_RemoteRayXGBoostActor pid=612, ip=10.0

(427,)


2021-12-15 00:02:58,173	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


(427, 2)


2021-12-15 00:02:58,935	INFO main.py:971 -- [RayXGBoost] Created 10 new actors (10 total actors). Waiting until actors are ready for training.
2021-12-15 00:03:01,175	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=19735) [00:03:01] task [xgboost.ray]:140123512409152 got new rank 0
(_RemoteRayXGBoostActor pid=19827) [00:03:01] task [xgboost.ray]:140234076301920 got new rank 2
(_RemoteRayXGBoostActor pid=19826) [00:03:01] task [xgboost.ray]:140698420431408 got new rank 1
(_RemoteRayXGBoostActor pid=704, ip=10.0.0.17) [00:03:01] task [xgboost.ray]:140432323750928 got new rank 3
(_RemoteRayXGBoostActor pid=705, ip=10.0.0.17) [00:03:01] task [xgboost.ray]:139777258818576 got new rank 4
(_RemoteRayXGBoostActor pid=861, ip=10.0.0.18) [00:03:01] task [xgboost.ray]:139846296669152 got new rank 5
(_RemoteRayXGBoostActor pid=860, ip=10.0.0.18) [00:03:01] task [xgboost.ray]:140087635639264 got new rank 6
(_RemoteRayXGBoostActor pid=826, ip=10.0.0.20) [00:0

(427,)


In [13]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(
        num_actors=10,  # Number of remote actors
        cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))


2021-12-15 00:02:11,779	INFO main.py:971 -- [RayXGBoost] Created 10 new actors (10 total actors). Waiting until actors are ready for training.
2021-12-15 00:02:14,038	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=549, ip=10.0.0.17) [00:02:14] task [xgboost.ray]:140446530374672 got new rank 3
(_RemoteRayXGBoostActor pid=18505) [00:02:14] task [xgboost.ray]:139997222566784 got new rank 0
(_RemoteRayXGBoostActor pid=18596) [00:02:14] task [xgboost.ray]:140666678270560 got new rank 1
(_RemoteRayXGBoostActor pid=18595) [00:02:14] task [xgboost.ray]:139792895871680 got new rank 2
(_RemoteRayXGBoostActor pid=674, ip=10.0.0.18) [00:02:14] task [xgboost.ray]:140170164730848 got new rank 4
(_RemoteRayXGBoostActor pid=673, ip=10.0.0.18) [00:02:14] task [xgboost.ray]:140613048720352 got new rank 5
(_RemoteRayXGBoostActor pid=610, ip=10.0.0.20) [00:02:14] task [xgboost.ray]:140270350560272 got new rank 6
(_RemoteRayXGBoostActor pid=550, ip=10.0.0.21) [00:0

Final training error: 0.0035


In [17]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 10
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors,
    cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-12-15 00:06:57,649	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-12-15 00:06:57,685	INFO logger.py:605 -- pip install "ray[tune]" to see TensorBoard files.
2021-12-15 00:06:57,685	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2021-12-15 00:06:57,703	WARNING tune.py:570 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


Trial name,status,loc,eta,max_depth,subsample
train_model_eb29e_00000,PENDING,,0.0393359,1,0.57664
train_model_eb29e_00001,PENDING,,0.000317791,8,0.885296
train_model_eb29e_00002,PENDING,,0.00621972,2,0.824497
train_model_eb29e_00003,PENDING,,0.0110035,3,0.570246


(ImplicitFunc pid=866, ip=10.0.0.20) 2021-12-15 00:06:59,785	INFO main.py:971 -- [RayXGBoost] Created 10 new actors (10 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=22893) 2021-12-15 00:07:00,215	INFO main.py:971 -- [RayXGBoost] Created 10 new actors (10 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=866, ip=10.0.0.20) 2021-12-15 00:07:02,358	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=806, ip=10.0.0.17) [00:07:02] task [xgboost.ray]:139956931050512 got new rank 0
(_RemoteRayXGBoostActor pid=807, ip=10.0.0.17) [00:07:02] task [xgboost.ray]:140015142816784 got new rank 1
(_RemoteRayXGBoostActor pid=809, ip=10.0.0.17) [00:07:02] task [xgboost.ray]:140486460611600 got new rank 2
(_RemoteRayXGBoostActor pid=808, ip=10.0.0.17) [00:07:02] task [xgboost.ray]:139929721162768 got new rank 3


Trial name,status,loc,eta,max_depth,subsample
train_model_eb29e_00000,RUNNING,10.0.0.13:22893,0.0393359,1,0.57664
train_model_eb29e_00001,RUNNING,10.0.0.20:866,0.000317791,8,0.885296
train_model_eb29e_00002,PENDING,,0.00621972,2,0.824497
train_model_eb29e_00003,PENDING,,0.0110035,3,0.570246


(ImplicitFunc pid=22893) 2021-12-15 00:07:02,884	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=22923) [00:07:02] task [xgboost.ray]:140088291420720 got new rank 0
(_RemoteRayXGBoostActor pid=22927) [00:07:02] task [xgboost.ray]:140535787313712 got new rank 4
(_RemoteRayXGBoostActor pid=22926) [00:07:02] task [xgboost.ray]:140507433106016 got new rank 3
(_RemoteRayXGBoostActor pid=22925) [00:07:02] task [xgboost.ray]:140169966602800 got new rank 2
(_RemoteRayXGBoostActor pid=22924) [00:07:02] task [xgboost.ray]:140184059836976 got new rank 1
(_RemoteRayXGBoostActor pid=897, ip=10.0.0.20) [00:07:02] task [xgboost.ray]:140358549548048 got new rank 5
(_RemoteRayXGBoostActor pid=896, ip=10.0.0.20) [00:07:02] task [xgboost.ray]:139644901964816 got new rank 4
(_RemoteRayXGBoostActor pid=1000, ip=10.0.0.18) [00:07:02] task [xgboost.ray]:140669231053840 got new rank 5
(_RemoteRayXGBoostActor pid=898, ip=10.0.0.20) [00:07:03] task [xgboost.ray]:14065597

...

Result for train_model_eb29e_00003:
  date: 2021-12-15_00-07-14
  done: false
  experiment_id: c07865d953a744a2899443cd2658c046
  hostname: 58ed1aa158f4456e8dc2bdb7ba9efb83000004
  iterations_since_restore: 1
  node_ip: 10.0.0.20
  pid: 1028
  time_since_restore: 6.072114706039429
  time_this_iter_s: 6.072114706039429
  time_total_s: 6.072114706039429
  timestamp: 1639526834
  timesteps_since_restore: 0
  train-error: 0.049209
  train-logloss: 0.684121
  training_iteration: 1
  trial_id: eb29e_00003
  
Result for train_model_eb29e_00002:
  date: 2021-12-15_00-07-15
  done: false
  experiment_id: be2b5b01d83344bd848c536dfd188e25
  hostname: nc6ic2
  iterations_since_restore: 1
  node_ip: 10.0.0.13
  pid: 23074
  time_since_restore: 6.861549377441406
  time_this_iter_s: 6.861549377441406
  time_total_s: 6.861549377441406
  timestamp: 1639526835
  timesteps_since_restore: 0
  train-error: 0.050967000000000005
  train-logloss: 0.688002
  training_iteration: 1
  trial_id: eb29e_00002
  
Res

(ImplicitFunc pid=1028, ip=10.0.0.20) 2021-12-15 00:07:15,268	INFO main.py:1495 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 6.36 seconds (3.77 pure XGBoost training time).


Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_eb29e_00000,TERMINATED,10.0.0.13:22893,0.0393359,1,0.57664,10,5.78597,0.484098,0.068541
train_model_eb29e_00001,TERMINATED,10.0.0.20:866,0.000317791,8,0.885296,10,7.11983,0.69037,0.01406
train_model_eb29e_00002,TERMINATED,10.0.0.13:23074,0.00621972,2,0.824497,10,7.07099,0.645231,0.045694
train_model_eb29e_00003,TERMINATED,10.0.0.20:1028,0.0110035,3,0.570246,10,6.36097,0.609653,0.024605


(ImplicitFunc pid=23074) 2021-12-15 00:07:15,420	INFO main.py:1495 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 7.07 seconds (3.54 pure XGBoost training time).
2021-12-15 00:07:15,560	INFO tune.py:626 -- Total run time: 17.91 seconds (17.73 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0003177912687455016, 'subsample': 0.8852956856563128, 'max_depth': 8}


### Shutdown when not used


In [ ]:
ray_on_aml.shutdown()


## Testing Ray on Job Cluster

In [10]:
ws = Workspace.from_config()

compute_cluster = 'worker-cpu-v3'
maxnode =5
vm_size='STANDARD_DS3_V2'
vnet='rayvnet'
subnet='default'
exp ='ray_on_aml_job'
ws_detail = ws.get_details()
ws_rg = ws_detail['id'].split("/")[4]
vnet_rg=None
try:
    ray_cluster = ComputeTarget(workspace=ws, name=compute_cluster)

    print('Found existing cluster, use it.')
except ComputeTargetException:
    if vnet_rg is None:
        vnet_rg = ws_rg
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                        min_nodes=0, max_nodes=maxnode,
                                                        vnet_resourcegroup_name=vnet_rg,
                                                        vnet_name=vnet,
                                                        subnet_name=subnet)
    ray_cluster = ComputeTarget.create(ws, compute_cluster, compute_config)

    ray_cluster.wait_for_completion(show_output=True)


rayEnv = Environment.from_conda_specification(name = "rayEnv",
                                             file_path = "../examples/conda_env.yml")

# rayEnv = Environment.get(ws, "rayEnv", version=19)


src = ScriptRunConfig(source_directory='../examples/job',
                script='aml_job.py',
                environment=rayEnv,
                compute_target=ray_cluster,
                distributed_job_config=PyTorchConfiguration(node_count=maxnode),
                    # arguments = ["--master_ip",master_ip]
                )
run = Experiment(ws, exp).submit(src)

Found existing cluster, use it.


In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
